In [1]:
#| default_exp core

In [2]:
%load_ext autoreload
%autoreload 2

# pipeline

In [3]:
#| notest
# Import specific functions we need from src, not everything
from ergativegpt.src import UtteranceClassification, get_metrics

In [4]:
import pandas as pd

from langchain.chains.openai_functions import create_structured_output_runnable
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from tqdm.notebook import tqdm

from sklearn.metrics import precision_recall_fscore_support,  accuracy_score

import altair as alt

from datetime import datetime

## load data

In [ ]:
#| notest
df = pd.read_excel('../in/Data_GPT_V2.xlsx')
df

,Token,Corpus,Source,Year,Variety,verbRealization,verbLemma,Transitivity,Transitivity_GPT,subjectAnimacy,subjectAnimacy_GPT,subjectRole,subjectRole_GPT,Construction,Construction_GPT
0,All the videos I'd been uploading to Facebook ...,NOW,https://www.vulture.com/article/marc-rebillet-...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
1,The investigation began after discovering the ...,NOW,https://www.herald-dispatch.com/news/police-ro...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
2,Elizabeth has been uploading one throwback bik...,NOW,https://hollywoodlife.com/2021/02/02/elizabeth...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
3,This tester community has been uploading video...,NOW,https://www.thedailybeast.com/tesla-has-nda-si...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
4,"Throughout the holiday, which Haim has dubbed ...",NOW,https://www.vanityfair.com/style/2021/12/adam-...,2021,AmE,uploading,upload,Transitive,NaN,Animate,NaN,Agent,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,These results will simultaneously upload to bl...,NOW,https://www.wfmz.com/news/pr_newswire/pr_newsw...,2021,AmE,upload,upload,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
196,Completed rides will automatically upload to a...,NOW,https://www.road.cc/content/review/264803-waho...,2019,BrE,upload,upload,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
197,It will automatically upload to your Digital D...,NOW,http://lifehacker.com/5920477/get-digital-doub...,2012,AmE,upload,upload,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN
198,"This way, all the lab has to do is scan the co...",NOW,https://www.sfgate.com/news/article/New-Mexico...,2021,AmE,upload,upload,Intransitive,NaN,Inanimate,NaN,Patient,NaN,1,NaN


## set up model

In [7]:
#| notest
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
#| notest
# model="gpt-3.5-turbo"
model="gpt-4"

llm = ChatOpenAI(model=model, temperature=0)

In [33]:
#| notest
with open('../in/prompt.md', 'r') as f:
        prompt_txt = f.read()

In [34]:
#| notest
prompt = ChatPromptTemplate.from_messages([
        ("system", prompt_txt),
        ("human", 'Please classify the following sentence: {input}')
])

In [35]:
#| notest
runnable = create_structured_output_runnable(UtteranceClassification, llm, prompt)

## run processing

In [36]:
#| notest

results_list = []

df_dev = (df
    # .sample(5)
)

for _, row in tqdm(df_dev.iterrows(), total=len(df_dev)):
    try:
        result = runnable.invoke({"input": row['verbRealization'] + ' in: ' + row['Token']})
        row_data = {
            'text': row['Token'],
            'transitivity': row['Transitivity'],
            'gpt_transitivity': result.gpt_transitivity,
            'causativity': row['Construction'],
            'gpt_causativity': result.gpt_causativity,
            'subject_animacy': row['subjectAnimacy'],
            'gpt_subject_animacy': result.gpt_subject_animacy,
            'subject_role': row['subjectRole'],
            'gpt_subject_role': result.gpt_subject_role,

            'gpt_subject': result.gpt_subject,
            'gpt_verb': result.gpt_verb,
            'gpt_object': result.gpt_object
        }
        results_list.append(row_data)
    except:
        continue

results = pd.DataFrame(results_list)

  0%|          | 0/200 [00:00<?, ?it/s]

## calculate metrics

align labels of factor levels between human and gpt labelling

In [37]:
#| notest
results['transitivity'] = results['transitivity'].replace({'Transitive': 'transitive', 'Intransitive': 'intransitive'})
results['causativity'] = results['causativity'].replace({0: 'causative', 1: 'anticausative'})
results['subject_animacy'] = results['subject_animacy'].replace({'Animate': 'animate', 'Inanimate': 'inanimate'})
results['subject_role'] = results['subject_role'].replace({'Agent': 'agent', 'Patient': 'patient'})

set variables and positive labels

In [38]:
#| notest
vars = {'transitivity': 'intransitive',
	'causativity': 'anticausative',
	'subject_animacy': 'inanimate',
	'subject_role': 'patient'}

calculate metrics

In [39]:
#| notest
metrics_vars = []
for var in vars.items():
	metrics_var = get_metrics(results, var[0], var[1])
	metrics_vars.append(metrics_var)

metrics = pd.concat(metrics_vars)

metrics

,variable,metric,score
0,transitivity,precision,0.960
1,transitivity,recall,0.680
2,transitivity,accuracy,0.725
3,transitivity,F1,0.800
0,causativity,precision,0.590
1,causativity,recall,0.270
2,causativity,accuracy,0.540
3,causativity,F1,0.370
0,subject_animacy,precision,0.990
1,subject_animacy,recall,0.870


plot metrics

In [40]:
chart = alt.Chart(metrics).mark_bar().encode(
	y='score:Q',
	x=alt.X('metric:N', sort=metrics['metric'].tolist()),
	color=alt.Color('metric', legend=None),
).facet(column='variable:N')

chart

alt.FacetChart(...)

## save results

In [47]:
datetime = datetime.now().strftime("%Y-%m-%d_%H:%M")

In [48]:
with open(f'../out/{datetime}_{model}_prompt.md', 'w') as f_prompt:
	f_prompt.write(prompt_txt)

In [49]:
chart.save(f'../out/{datetime}_{model}_metrics.png', scale=2.0)

In [50]:
results.to_csv(f'../out/{datetime}_{model}_data.csv', index=False)

In [51]:
metrics.to_csv(f'../out/{datetime}_{model}_metrics.csv', index=False)